# 최종 LSTM 모델로 진행

# 라이브러리 및 데이터

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import random as rn

from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

rc('font', family='AppleGothic')
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

warnings.filterwarnings(action='ignore')

In [3]:
import argparse
import random as rn
import tensorflow as tf
import keras.layers as L

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error, r2_score


In [5]:
#재현성을 위한 seed 설정
seed_num = 42
np.random.seed(seed_num)
rn.seed(seed_num)
tf.random.set_seed(seed_num)

In [6]:
data = pd.read_csv('./raw_data/total_day.csv', index_col = 0)
weathers = ['평균기온', '일강수량', '일사량', '미세먼지'] # 종속변수
products = data.columns[4:].to_list()

# Model

In [7]:
def build_data(data, cat):
    cols = [cat]+ weathers
    X = data[cols].reset_index(drop=False)
    X.rename(columns={cat:'y'}, inplace=True)

    X.index = X['date']
    del X['date']

    return X

In [8]:
def minmax_scalar(X):
    idx = X.index
    col = X.columns

    scalar = MinMaxScaler()
    scaled_X = pd.DataFrame(scalar.fit_transform(X))
    scaled_X.index = idx
    scaled_X.columns = col

    return scaled_X

In [9]:
def split_xy(dataset, time_steps, y_column): 
    
    x, y = list(), list()
    
    for i in range(len(dataset)): # 2년치 일별데이터면 730번 for문 실행
        x_end_number = i + time_steps
        y_end_number = x_end_number + y_column

        if y_end_number > len(dataset): # 데이터 끝에 다다르면 끝
            break

        tmp_x = np.array(dataset)[i:x_end_number, 1:] # 1:으로 수정(y칼럼 제외)
        tmp_y = np.array(dataset)[x_end_number:y_end_number, 0] # 0으로 수정(y칼럼)
        
        x.append(tmp_x)
        y.append(tmp_y)
        
    return np.array(x), np.array(y)

In [10]:
def data_pipeline(data, cat, time_steps, y_columns):
    data = build_data(data, cat)

    min = data['y'].min()
    max = data['y'].max()

    X = minmax_scalar(data)
    Xy = X.dropna()

    X, y = split_xy(Xy, time_steps, y_columns)

    X_train, y_train = X[:-7],y[:-7]
    X_test, y_test = X[-7:],y[-7:]

    # X_train.shape[2] = feature 개수
    X_test=X_test.reshape(-1, time_steps, X_train.shape[2]) 
    y_test=y_test.reshape(-1, y_columns)


    return X_train, y_train, X_test, y_test, min, max

In [11]:
def LSTM_(data, cat, timestep=7, y_column=1):
    X_train, y_train, X_test, y_test, min, max = data_pipeline(data, cat, timestep, y_column)
    
    model = Sequential()
    model.add(LSTM(128, input_shape = (None, X_train.shape[2])))
    model.add(Dense(16))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse')
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
    
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stopping], validation_data = (X_test, y_test))

    y_pred = model.predict(X_test, batch_size=1)
    
    r2 = r2_score(y_test, y_pred)
    mse = np.mean((y_test-y_pred)**2)
    
#     rmse = np.sqrt(mse)
#     mae = mean_absolute_error(y_test, y_pred)
#     rmsle = RMSLE_fun(np.array(y_test), np.array(y_pred))
    
    return model, r2, mse

# model predict

In [12]:
cat = '공기청정기'
LSTM_model, LSTM_r2, LSTM_mse = LSTM_(data, cat)

In [13]:
import shap

# we use the first 100 training examples as our background dataset to integrate over
explainer = shap.DeepExplainer(LSTM_model, X_train[:100])

# explain the first 10 predictions
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(X_test[:10])

ModuleNotFoundError: No module named 'shap'

In [ ]:
# init the JS visualization code
shap.initjs()

# transform the indexes to words
words = imdb.get_word_index()
num2word = {}
for w in words.keys():
    num2word[words[w]] = w
x_test_words = np.stack([np.array(list(map(lambda x: num2word.get(x, "NONE"), x_test[i]))) for i in range(10)])

# plot the explanation of the first prediction
# Note the model is "multi-output" because it is rank-2 but only has one column
shap.force_plot(explainer.expected_value[0], shap_values[0][0], x_test_words[0])